# Floor space

In [ ]:
import folium
import geopandas as gpd
import pandas as pd
from libpysal import graph

In [ ]:
cd /home/lisa/work/people_places_germany/code

In [ ]:
from clusters import assign_clusters
from spatial_autocorrelation import lisa

In [ ]:
clusters = gpd.read_parquet(
    "/data/cluster_data/clusters_umap_freiburg_100_3_gaussian_euclidean_complete_chebyshev_3.pq"
)

In [ ]:
clusters

In [ ]:
# clusters.explore(column=clusters.index,cmap='tab20')

# Raw Data

In [ ]:
demographics = pd.read_parquet(
    "/data/processed_data/Wohnungen100m.parquet"
).reset_index()

In [ ]:
gdf, largest_overlap, data = assign_clusters(demographics, clusters)

In [ ]:
gdf["FLOOR_SPACE"]

In [ ]:
gdf["STATS", "Average floor space"] = (
    data["FLOOR_SPACE"]["Under 30"].fillna(0) * 20
    + data["FLOOR_SPACE"]["30 - 39"].fillna(0) * 34.5
    + data["FLOOR_SPACE"]["40 - 49"].fillna(0) * 44.5
    + data["FLOOR_SPACE"]["50 - 59"].fillna(0) * 54.5
    + data["FLOOR_SPACE"]["60 - 69"].fillna(0) * 64.5
    + data["FLOOR_SPACE"]["70 - 79"].fillna(0) * 74.5
    + data["FLOOR_SPACE"]["80 - 89"].fillna(0) * 84.5
    + data["FLOOR_SPACE"]["90 - 99"].fillna(0) * 94.5
    + data["FLOOR_SPACE"]["100 - 109"].fillna(0) * 104.5
    + data["FLOOR_SPACE"]["110 - 119"].fillna(0) * 114.5
    + data["FLOOR_SPACE"]["120 - 129"].fillna(0) * 124.5
    + data["FLOOR_SPACE"]["130 - 139"].fillna(0) * 134.5
    + data["FLOOR_SPACE"]["140 - 149"].fillna(0) * 144.5
    + data["FLOOR_SPACE"]["150 - 159"].fillna(0) * 154.5
    + data["FLOOR_SPACE"]["160 - 169"].fillna(0) * 164.5
    + data["FLOOR_SPACE"]["170 - 179"].fillna(0) * 174.5
    + data["FLOOR_SPACE"]["180 and more"].fillna(0) * 200
) / data["FLOOR_SPACE"].sum(axis=1)

In [ ]:
gdf1 = gdf[["ID", "geometry", "STATS", "cluster"]]
gdf1.columns = gdf1.columns.get_level_values(0)

In [ ]:
gdf2 = gdf[["ID", "FLOOR_SPACE"]]
gdf2.columns = gdf2.columns.get_level_values(1)

In [ ]:
gdf1 = gdf1.merge(gdf2, how="left", on="ID")

In [ ]:
# gdf1.cx[4150000:4170000,2760000:2780000].explore("STATS", cmap="coolwarm", prefer_canvas=True)

# Compute Spatial Lag

In [ ]:
gdf1 = gdf1.cx[4150000:4170000, 2760000:2780000]

In [ ]:
gdf1 = gdf1[gdf1.cluster.notna()]

In [ ]:
queen = graph.Graph.build_contiguity(gdf1, rook=False)

In [ ]:
row_wise_queen = queen.transform("R")

In [ ]:
for i in [
    "STATS",
    "50 - 59",
    "60 - 69",
    "80 - 89",
    "70 - 79",
    "90 - 99",
    "110 - 119",
    "40 - 49",
    "180 and more",
    "30 - 39",
    "Under 30",
    "120 - 129",
    "100 - 109",
    "140 - 149",
    "150 - 159",
    "130 - 139",
    "160 - 169",
    "170 - 179",
]:
    gdf1.loc[:, i + "_lag"] = row_wise_queen.lag(gdf1[i])

In [ ]:
gdf1

In [ ]:
gdf1.to_parquet(
    "/home/lisa/work/people_places_germany/Notebooks/temp_data/floorspace.parquet"
)

# Local Spatial Autocorrelation

In [ ]:
mi, gdf_05 = lisa(gdf1.cx[4150000:4170000, 2760000:2780000], "STATS", 0.05)

In [ ]:
print(f"Moran's I: {mi.I}, p-value: {mi.p_sim}")

In [ ]:
# gdf_05.explore("sig_cluster", prefer_canvas=True, cmap=["#d7191c","#fdae61","#abd9e9","#2c7bb6","lightgrey"])

# Cluster Aggregation

In [ ]:
d = []
for i in range(0, int(gdf1["cluster"].max())):
    d.append(
        {
            "stats": gdf1[gdf1["cluster"] == i]["STATS"].mean(),
            "stats count": gdf1[gdf1["cluster"] == i]["STATS"].count(),
        }
    )

stats = pd.DataFrame(d)

In [ ]:
stats["label"] = stats.index
stats = clusters.merge(stats, on="label")

In [ ]:
# stats.explore(column='stats', cmap="coolwarm", prefer_canvas=True)

In [ ]:
m = stats.explore(
    column="stats", cmap="coolwarm", prefer_canvas=True, name="cluster aggregation"
)

gdf_05.explore(
    "sig_cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
    name="local spatial autocorrelation",
    m=m,
)

gdf1.cx[4150000:4170000, 2760000:2780000].explore(
    "STATS", cmap="coolwarm", prefer_canvas=True, name="raw data", m=m
)
gdf1.cx[4150000:4170000, 2760000:2780000].explore(
    "STATS_lag", cmap="coolwarm", prefer_canvas=True, name="raw data lag", m=m
)

# Add a layer control to toggle the layers on and off
folium.LayerControl().add_to(m)

# Display the map
m